In [2]:
import numpy as np
from pyspectrum import Spectrum, Data

ImportError: DLL load failed while importing _pyspectrum: The specified module could not be found.

In [ ]:
class MockSpectrometer:
    def __init__(self):
        self.__wavelengths = np.linspace(400, 700, 1000)  # Пример калибровки длины волн
        self.__dark_signal = None
        self.__config = None

    def set_config(self, exposure, n_times, wavelength_calibration_path):
        self.__config = {
            'exposure': exposure,
            'n_times': n_times,
            'wavelength_calibration_path': wavelength_calibration_path
        }
        print(f"Mock device configured: exposure={exposure}, n_times={n_times}, "
              f"wavelength_calibration_path={wavelength_calibration_path}")

    def read_dark_signal(self, n_times):
        # Подделываем темновой сигнал как случайные данные
        self.__dark_signal = Data(
            intensity=np.random.normal(0, 0.01, (n_times, 1000)),
            clipped=False,
            exposure=self.__config['exposure']  # Добавляем экспозицию
        )
        print(f"Mock dark signal read with {n_times} frames.")

    def read_raw(self):
        # Подделываем необработанный сигнал как случайные данные
        return Data(
            intensity=np.random.random(1000) * 1000,  # Имитация сырых данных
            clipped=False,
            exposure=self.__config['exposure']  # Добавляем экспозицию
        )

    def read(self, force: bool = False) -> Spectrum:
        if self.__wavelengths is None and not force:
            raise Exception('Wavelength calibration is not loaded')
        if self.__dark_signal is None:
            raise Exception('Dark signal is not loaded')

        data = self.read_raw()
        scale = 1.0  # Для простоты масштаб делаем равным 1
        return Spectrum(
            intensity=(data.intensity / scale - np.round(
                np.mean(self.__dark_signal.intensity / scale, axis=0))) * scale,
            clipped=data.clipped,
            wavelength=self.__wavelengths,
            exposure=self.__config['exposure'],
        )


In [ ]:
# Используем поддельное устройство
mock_device = MockSpectrometer()

# Настройка поддельного устройства
mock_device.set_config(
    exposure=1,     # Экспозиция 1 мс
    n_times=1000,   # За один раз будет считано 1000 кадров
    wavelength_calibration_path='data/profile.json'
)

# Чтение темнового сигнала
mock_device.read_dark_signal(n_times=1000)

# Чтение поддельных данных с устройства
mock_spectrum = mock_device.read()

# Сохранение поддельных данных (можно сделать реальный save метод или просто вывести)
print("Mock spectrum data:", mock_spectrum.intensity)
